<img src="https://callysto.ca/wp-content/uploads/2018/06/Callysto_Notebook-Banner_Top_06.06.18.jpg" alt="Drawing" style="width: 1800px;"/>

In [ ]:
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
import numpy as np
import math
import ipywidgets as widgets
from IPython.display import display, Math, Latex, HTML, IFrame
from astropy.table import Table, Column
from ipywidgets import interact, interactive

py.offline.init_notebook_mode(connected=True)
%matplotlib inline

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 14}

plt.rc('font', **font)

'''Above, we are importing all the necessary modules in order to run the notebook. 
Numpy allows us to define arrays of values for our variables to plot them
matplotlib is what we use to create the figures
the display and widgets are to make the notebook look neat
'''

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')
        
    

<h1><center> Interpreting American Sign Language Using a Leap Motion Device </center></h1>

<img src="http://www.richmedia.com/richideas/img/how-the-leap-motion-controller-could-change-the-world-banner.jpg" alt="Drawing" style="width: 500px;"/>


## Introduction

In this notebook, we will go over a program designed by an Albertan highschool student that aims to interpret American Sign Language (ASL) symbols. Currently, the program is able to accurately interpret half of the ASL alphabet.

This project involved the use of a Leap Motion Controller (LMC). The Leap Motion Controller is a device that is able to track the movement of the hands, and the program uses this data to recognize when the hands have made an official American Sign Language (ASL) symbol. When this happens, the program prints the symbol that has been recognized. 

The code for this program was written in Python, but if you are unfamiliar with computer coding, do not be discouraged! The student that created this program had minimal coding experience when the project began, and learned all of the necessary concepts along the way.

This lesson will go through the concepts that made the project possible, and will not explicitly outline every line of code.

Before we get into how the project began, here is a little background on ASL and the LMC.

### American Sign Language

American Sign Language (ASL) was develped in order to enable deaf and mute individuals to communicate. In ASL, there are 26 unique hand symbols and getures that correspond to the 26 letters of the English alphabet.

<img src="https://cdn8.bigcommerce.com/s-dc9f5/images/stencil/original/products/2095/8082/ASL_Trans_ABCs__08283.1443708103.jpg?c=2&imbypass=on" alt="Drawing" style="width: 500px;"/>

A person that communicates with ASL will not use these symbols to spell out every word they are trying to say, however, they act as a good starting point when building a program to recognize distinct symbols. To learn more about ASL, check out the following link: https://www.nidcd.nih.gov/health/american-sign-language.

### The Leap Motion Controller (LMC)

The Leap Motion Controller was developed by Leap Motion Inc. in 2012. It is a small, portable device that uses a combination of infrared cameras and LEDs to track the motion of someones hands and fingers, without needing to touch or manipulate the device in any way. The controller can be plugged into any computer or laptop, and can also be connected to a virtual reality (VR) headset. To get a better idea of how the LMC is able to track the motion of the hands, check out the following video:

In [ ]:
%%html
<center><iframe width="560" height="315" align="middle" src="https://www.youtube.com/embed/pqeWzp6weMM?rel=0&amp;showinfo=0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe><center>

<center> Video from https://www.youtube.com/watch?time_continue=36&v=pqeWzp6weMM, August 13th, 2018. </center>
<center> Figure 1: Here we see a demontstration of how the Leap Motion Controller is able to digitally visualize and track the movement of the hands, including the knuckles, joints, fingertips and palms.</center> 

As you can see above, the LMC creates a digital representation of the hands in a 3-dimensional space. Much like a video camera, the LMC records the position of the hands frame by frame (between 20 and 200 times per second). When connected to a computer, this data can be accessed and manipulated. It can accurately track two hands at once, but for this project, we will only focus on using one hand.

### Procedure
We will now go through the procedural steps that led to the creation of the program.

1. Establishing a Goal
2. Gathering Data
3. Interpreting Data - Creating an Algorithm
4. Generating Output

#### Establishing a Goal

The initial idea for the project was to develop a program that would be able to translate American Sign Language using the Leap Motion Controller. However, we did not expect to have a fully successful ASL translator within the time we had.
Instead, we decided to focus on a simpler task: designing a program that uses the LMC to interpret individual *letters* of the ASL alphabet. As there are 26 distinct symbols in the ASL alphabet, this provided us with a challenge that was attainable. If successful, we would also be taking the first steps towards a fully functional ASL translator.

**Goal: design a program that uses data from the LMC to interpret the ASL alphabet.**


#### Gathering Data

To begin our project, we needed to access and analyze the data from the LMC. On the Leap Motion website, you can download Software Development Kits (SDKs) that will allow you to access data from the LMC on the platform of your choice. We connected the LMC to a computer and chose to use the Python coding platform to work with the data. 

As seen in the image below, the LMC measures coordinates with respect to the center of the device itself.

<img src="https://di4564baj7skl.cloudfront.net/documentation/images/Leap_Axes.png" alt="Drawing" style="width: 500px;"/>
<center> Figure ??: This is how the LMC measures coordinates for the hand. </center>

The LMC is able to provide coordinates for many individual component of the hand.

<img src="https://preview.ibb.co/bzqC3K/visualizer.png" alt="Drawing" style="width: 400px;"/>
<center> Iamge taken from https://sayagusti.wordpress.com/2014/10/08/konfigurasi-leap-motion-dengan-unity-free/ on August 23rd, 2018. </center>
<center> Figure ??: These are some of the bones in the hand that the LMC is able to track and provide coordinates for. </center>

The coordinates for these components of the hand are displayed on the screen frame by frame.

<img src="https://preview.ibb.co/mvzcLz/Screen_Shot_2018_08_22_at_11_39_20_AM.png" alt="Drawing" style="width: 700px;"/>
<center> Figure ??: This screenshot shows how data from the LMC is displayed on a computer. The end point of the distal bone is used as the fingertip coordinate. </center>

We had to figure out a way we could use this data to accomplish our goal. What could we look for that would allow us to distinguish one ASL symbol from another?

It may seem obvious, but it is the position and orientation of each finger that separates one ASL letter from another. However, the coordinates given by the LMC alone do not tell us anything about the orientation of the fingers. The coordinates just tell us where each component of the hand is inside the 3D space that the LMC can see. We need to know where the fingers are *relative to each other*. More specifically, we need to know the relative distances between the individual fingertips, and the distance from each fingertip to the palm.


> For example, the ASL symbol for "S" shows the four fingers curled into a fist, with the thumb pointing towards the palm. To recognize this symbol, the program needs to know that all four fingers are bent down, and that the thumb is tucked towards the palm. This means that both the fingertip-to-palm distance and fingertip-to-fingertip distance for each finger needs to be measured. 

With these relative distances, we will be able to know if a finger is bent, upright, or touching another finger.

These values will provide us with the information we need in order to distinguish one ASL letter from another. We now need our Python program to take the data from the LMC and find the fingertip-to-fingertip distances, as well as the fingertip-to-palm distances.

#### Measuring Fingertip-to-Fingertip and Fingertip-to-Palm

Using the coordinate data that the LMC provides us, along with some basic trigonometry, we can find each fingertip-to-fingertip (FTF) and fingertip-to-palm (FTP) distance.

The coordinates for each fingertip, as well as the palm, have an $x$, $y$ and $z$ component. The distance between any two points in a three dimensional space can be shown as:

\begin{equation} 
Distance(P_1,P_2) = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2 + (z_2-z_1)^2}
\end{equation}

To calculate the FTF and FTP distances, we created a Python program that takes the $x$, $y$, and $z$ coordinates for each fingertip and the palm, and uses the above formula.

\begin{array}{| c | c |}
\hline
 \rm Fingertip\: to\: Palm\: Distances\: & \rm Fingertip\: to\: Fingertip\: Distances \\
  \hline
  Thumb\: to\: Palm & Thumb\: to\: Index \\\hline
  Index\: to\: Palm & Thumb\: to\: Middle \\\hline
  Middle\: to\: Palm & Thumb\: to\: Ring \\\hline
  Ring\: to\: Palm & Thumb\: to\: Pinky \\\hline
  Pinky\: to\: Palm & Index\: to\: Middle \\\hline
   & Index\: to\: Ring \\\hline
   & Index\: to\: Pinky \\\hline
   & Middle\: to\: Ring \\\hline
   & Middle\: to\: Pinky \\\hline
   & Ring\: to\: Pinky \\\hline
\end{array}
<center> The table shown above displays all of the FTF and FTP distances that are calculated by our Python program.</center>


#### Recognizing Symbols

Now that we have the proper set of data, the program needs to recognize when the orientation of the users hand forms an official ASL letter.

To simplify the way our program recognizes symbols, we established a binary system to represent the distinguishing features of each ASL letter, which are:
- which fingers are "down" or "up"
- which fingertips are touching each other

For example, the letter "A" involves all four fingers being bent down towards the palm, with only the thumb in the "up" position. The orientation for this letter is then:

<img src="https://preview.ibb.co/dLup0z/Orientation.png" alt="Drawing" style="width: 500px;"/>
<center> Figure ??: This group of 1's and 0's is what we used to identify the orientation of each ASL letter. This code can be thought of as the unique fingerprint for the letter "A". </center>

In the above image:
- "0" signifies that a finger is "down", or that two fingertips are touching
- "1" signifies that a finger is "up", or that two fingertips are not touching


The program is able to determine if a finger is up or down by measuring its FTP distance. If this distance is less than a certain value, the finger is considered to be "down". The same method is used to determine if two fingers are touching, but with the FTF distance being measured.


Much like an answer key for a test, we recorded the codes for the proper orientation of each letter. For example, the ASL symbol for the letter "A" would be made in front of the LMC, and the resulting 9 digit code would be manually recorded. This process was done for as many of the ASL letters as possible, and the information for each letter was stored in what is called a dictionary. 

In computer science, a "dictionary" refers to a type data storage, where each entry has two components: a key and a value. The reason why dictionaries are so useful is because they allow you to bind a value to a unique key. Therefore, when you lookup a particular key, the dictionary returns its value to you. 

You can think of a dictionary as a list that contains entries in the following format

\begin{equation}
\rm Dictionary = ( (Key\: 1 : Value\: 1), (Key\: 2 : Value\: 2), (Key\: 3 : Value\: 3) )
\end{equation}

The reason why a dictionary is useful in this scenario is because it allows each ASL letter to be bound to its coded orientation.

Use the following code segment to increase your understanding of how a dictionary works.

In [ ]:
def maker():
    #a function that lets you look up values in a dictionary
    letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    dct = dict()
    for i in range(len(letters)):
        dct[letters[i]] = 'Orientation for ' + '\'' + letters[i] + '\''

    return dct

def finder(key):
    y = maker()
    x = y[str(key)]
    print(x)

try:
    interact(finder, key = widgets.Text(placeholder =' ' , description = 'String',disabled = True));
except:
    pass
    

#### Output of Successful Translation

With a dictionary containing all of the correct hand orientations, the program now knows what coded orientations represent official ASL letters. Now, when a user is holding their hand in front of the LMC, the program takes the coordinate data, turns it into a 9 digit code, and checks if this code matches any of the codes in the saved dictionary. If there is a match, the program prints out the corresponding letter! If there is not a match, the program simply continues to check data from the LMC. 

Take a look at the flow chart below that summarizes how the program operates.

<img src="https://preview.ibb.co/j6gc2e/Screen_Shot_2018_08_22_at_7_58_01_PM.png" alt="Drawing" style="width: 600px;"/>
<center> Figure ??: This flow chart outlines how data from the Leap Motion Controller (LMC) is used to determine if the users hand is in the shape of an Official American Sign Language (ASL) letter. </center>



## Results

Using the methods outlined above, the program is able to accurately recognize 13 of the 26 ASL letters. These letters are A, B, C, F, I, L, O, P, U, V, W, X, and Y.

<img src="https://preview.ibb.co/gSKphe/Screen_Shot_2018_08_22_at_8_26_43_PM.png" alt="Drawing" style="width: 500px;"/>
<center> Figure ??: The Confusion Matrix shown above exemplifies the accuracy of the program. It shows how many times the program was able to correctly interpret each ASL letter out of 50 trials. </center>


## Future Improvements

There are a few reasons why some letters are not recognizable by the program:

- The LMC is only able to track the positions of the fingers if they are immediately visible to the camera. For instance, the symbol for the letter M requires the thumb to be placed underneath the other fingers. When this happens, the LMC cannot measure the coordinates of the thumb because it is hidden. Letters such as "N", "T", and "Q" cause the same problem.

- The program is not built to accomodate motion. Our program only measures static coordinates, and cannot recognize the specific gestures required to make symbols like "J" and "Z".

Solving these two issues would enable the program to begin recognizing more complex symbols, and go beyond just the ASL alphabet. ASL combines many dynamic hand gestures when forming sentences and phrases, and would require a much more observant program to translate them.


## Conclusion

This project is just one example of how a highschool student can use basic mathematics and simple computer coding to accomplish extraordinary things. Challenges such as these are an excellent way for students to learn different coding languages, and apply some of the concepts that they learn in class. 


<img src="https://callysto.ca/wp-content/uploads/2018/07/Callysto-Notebook-Banner_Bottom_07.30.18.jpg" alt="Drawing" style="width: 1800px;"/>